# Collecte de données


In [8]:
import os
import pandas as pd

# === Récupération du type de fichier image associé à chaque pokémon ===
lst_files = os.listdir('images/images')
lst_files = [f.split('.') for f in lst_files]
dico_files = {paire[0]: paire[1] for paire in lst_files}

#importation des données depuis un fichier csv
with open('images/pokemon.csv', 'r') as csv_file:

    # === Pré-traitement des données ===
    lines = csv_file.readlines()
    lines = lines[1:] #suppression de la première ligne (entête)
    
    for i,data in enumerate(lines):
        data = data.replace('\n', '')
        lines[i] = data.split(',')
        if len(lines[i]) == 2: #si le pokémon n'a qu'un seul type
            lines[i].append(None) #ajout d'une colonne vide pour le second type
        #récupération du type de fichier image associé   
        lines[i].append(dico_files[lines[i][0]])
        
    # === Création d'un DataFrame ===      
    df = pd.DataFrame(lines, columns=["name", "type1", "type2","img_type"])
    
    csv_file.close()

# Export du DataFrame vers un fichier JSON
df.to_json('donnees.json',orient='index')

# Étiquetage et annotation

In [ ]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans

imgfile = Image.open("images/images/pichu.png")

numarray = numpy.array(imgfile, numpy.uint8)

numarray = numarray.reshape(-1, 9)

n_clusters = 4
clusters = KMeans(n_clusters=n_clusters, n_init=20)
clusters.fit(numarray)

couleurs = []
for i in range(len(clusters.cluster_centers_)):
    print(clusters.cluster_centers_[i])
    couleurs.append([numpy.sum(clusters.labels_==i),"#%02x%02x%02x" % (
        math.ceil(clusters.cluster_centers_[i][0]),
        math.ceil(clusters.cluster_centers_[i][1]),
        math.ceil(clusters.cluster_centers_[i][2]),
    )])
    
print (couleurs)

# Analyses de données

In [27]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import json
import random

# Fonction pour charger et afficher une image aléatoire
def afficher_image_aleatoire():
    # Charger les données à partir du fichier JSON
    with open("donnees.json", "r") as f:
        donnees = json.load(f)

    # Sélectionner une image aléatoire
    image_id = str(random.randint(0, len(donnees) - 1))
    nom_image = donnees[image_id]["name"]
    img_type = donnees[image_id]["img_type"]

    # Chemin complet de l'image
    chemin_image = os.path.join("images/images", f"{nom_image}.{img_type}")

    # Chargement de l'image sélectionnée
    image = Image.open(chemin_image)

    # Conversion de l'image en format compatible avec Tkinter
    image_tk = ImageTk.PhotoImage(image)

    # Affichage de l'image dans la fenêtre Tkinter
    label_image.config(image=image_tk)
    label_image.image = image_tk

    # Affichage du nom de l'image
    label_nom_image.config(text=nom_image)

#fonction pour remplacer le bouton "Afficher une image" par les boutons "J'aime" et "Je n'aime pas"
def remplacement_bouton():
    bouton_afficher_image.pack_forget()
    bouton_jaime.pack(side=tk.LEFT, padx=5)
    bouton_jaime_pas.pack(side=tk.RIGHT, padx=5)
    afficher_image_aleatoire()

# Fonction pour les actions "J'aime" et "Je n'aime pas"
def action_jaime():
    afficher_image_aleatoire()

def action_jaime_pas():
    afficher_image_aleatoire()

# Création de la fenêtre principale
root = tk.Tk()
root.title("Affichage d'une image aléatoire depuis un fichier JSON")

# Création d'une étiquette pour afficher l'image
label_image = tk.Label(root)
label_image.pack(padx=10, pady=10)

# Création d'une étiquette pour afficher le nom de l'image
label_nom_image = tk.Label(root, font=("Helvetica", 12))
label_nom_image.pack(pady=5)

# Création de deux boutons pour "J'aime" et "Je n'aime pas"
bouton_jaime = tk.Button(root, text="J'aime", command=action_jaime)
bouton_jaime_pas = tk.Button(root, text="Je n'aime pas", command=action_jaime_pas)

# Création d'un bouton pour afficher une nouvelle image
bouton_afficher_image = tk.Button(root, text="Afficher une image", command=remplacement_bouton)
bouton_afficher_image.pack(pady=10)

# Boucle principale
root.mainloop()


# Visualisation des données

In [ ]:
pass

# Système de recommandation

In [ ]:
pass

# Tests

pass